### <span style="color:darkgreen">**Riiid - GRU**</span>

In [ ]:
# IMPORTS

import os
import pickle

import torch

# my packages
from dataset import RiiidData
from model_gru import SaintNikolaGRU
from train import train_model
from utils import seed_everything, get_ma

# dataset config
from config_gru import max_quest, num_quest, start_response_token, sequel_response_token, batch_size
# model config
from config_gru import input_size_gru, hidden_size_gru, num_layers_gru
# training config
from config_gru import lr, epochs, warmup_steps

from config_gru import random_seed
seed_everything(random_seed)

# visible GPU card
gpu_idx = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_idx}"
device = torch.device("cuda")

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
# # FORMAT THE DATA - DO ONLY ONCE

# import pickle
# import pandas as pd
# from utils import csv_to_dict

# df = pd.read_parquet("../data/riiid_train.parquet")
# questions_container = csv_to_dict(df)

# # save files
# with open("./questions_container.pickle", "wb") as handle:
#     pickle.dump(questions_container, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
%%time

# load data container
with open("../models/riiid_container/questions_container.pickle", "rb") as handle:
    questions_container = pickle.load(handle)


In [ ]:
# TRAIN AND VALIDATION DATALOADERS
data = RiiidData(questions_container, max_quest, num_quest, start_response_token, sequel_response_token, batch_size)
data.sampling_process()
train_dataloader, val_dataloader = data.get_dataloaders(val_size=0.2)


In [ ]:
# INITIALIZE A MODEL INSTANCE
model = SaintNikolaGRU(device, num_quest, max_quest, input_size_gru, hidden_size_gru, num_layers_gru)
model.init_weights()
model.to(device)
print(f"The model has {model.num_parameters()} of trainable parameters!")

In [ ]:
# TRAIN THE MODEL
model, train_loss = train_model(model, device, train_dataloader, val_dataloader, lr, epochs, warmup_steps)

In [ ]:
# PLOT MOVING AVERAGE TRAIN LOSS
ma_loss = get_ma(train_loss, window_size=40)
plt.rcParams["figure.figsize"] = (20, 5)
plt.plot(ma_loss[100:])
plt.hlines(0.50, xmin=0, xmax=len(ma_loss[100:]), colors="r");